In [ ]:
import polars as pl
import polars.selectors as cs
from polars_ds.pipeline import Pipeline, Blueprint

# Builtin Pipeline Functions

To run this demo: pip install polars_ds>=0.4.5

In [ ]:
df = pl.read_parquet("../examples/dependency.parquet")
df.head()

In [ ]:
df.select(
    pl.col("Loan_Period").null_count()
)

In [ ]:
df.shape

In [ ]:
import os

os.environ["POLARS_VERBOSE"] = "1"

In [ ]:
import polars_ds as pds

df.select(
    pds.query_lstsq("Var1", "Existing_EMI", target = "Loan_Period", skip_null=True)
)

In [ ]:
# Create a blueprint first. 
# A blueprint is a plan for a pipeline. No hard work will be done until the blueprint is materialized, which
# is when the tranforms are fitted (e.g. scale learns the mean and std from base data)
# If target is specified for the blueprint, target will be excluded from all transformations that require a fit,
# and target will be auto-filled if the transformation requires a target field and when no target field is explicitly given.

bp = (
    Blueprint(df, name = "example", target = "approved") # You can optionally put target of the ML model here
    # Select only the columns we need
    .lowercase() # lowercase all columns
    .select(cs.numeric() | cs.by_name(["gender", "employer_category1", "city_category"]))
    # explicitly put target, since this is not the target for prediction. 
    # Use a linear regression with x1 = var1, x2=existing_emi to predict missing values in loan_period
    .linear_impute(features = ["var1", "existing_emi"], target = "loan_period") 
    .impute(["existing_emi"], method = "median")
    .append_expr( # generate some features
        pl.col("existing_emi").log1p().alias("existing_emi_log1p"),
        pl.col("loan_amount").log1p().alias("loan_amount_log1p"),
        pl.col("loan_amount").sqrt().alias("loan_amount_sqrt"),
    )
    .scale( # target is numerical, but will be excluded automatically because bp is initialzied with a target
        cs.numeric().exclude(["var1", "existing_emi_log1p"]), method = "standard"
    ) # Scale the columns up to this point. The columns below won't be scaled
    .append_expr(
        # Add missing flags
        pl.col("employer_category1").is_null().cast(pl.UInt8).alias("employer_category1_is_missing")
    )
    .one_hot_encode("gender", drop_first=True)
    .woe_encode("city_category") # No need to specify target because we initialized bp with a target
    .target_encode("employer_category1", min_samples_leaf = 20, smoothing = 10.0) # same as above
)

print(bp)

In [ ]:
# Materialize the blueprint
pipe:Pipeline = bp.materialize()
# Text representation of the pipeline
pipe

In [ ]:
df_transformed = pipe.transform(df)
df_transformed.head()

# Serialization Methods

Pickle + JSON support.

In [ ]:
import pickle
# The pipe object can be pickled
with open("pipe.pickle", "wb") as f:
    pickle.dump(pipe, f)

In [ ]:
with open("pipe.pickle", "rb") as f:
    pipe2 = pickle.load(f)

pipe2

In [ ]:
df_transformed_2 = pipe2.transform(df)
df_transformed_2

In [ ]:
from polars.testing import assert_frame_equal
# True
assert_frame_equal(df_transformed, df_transformed_2)

In [ ]:
# To save the pipeline as JSON

pipe.to_json("test.json")
pipe3 = Pipeline.from_json("test.json")
# True
assert_frame_equal(df_transformed, pipe3.transform(df))

# Custom Tranformations in Pipeline

Need version >= v0.4.6 (Not released yet)

In [ ]:
df = pl.read_parquet("../examples/dependency.parquet")
df.head()

In [ ]:
from typing import Union, List

# Any custom function must satistfy the following function signature:
# func(df:Union[pl.DataFrame, pl.LazyFrame], cols: List[str], ...) -> List[pl.Expr]
# where ... means kwargs
# Here is a custom imputer

def smallest_abs_impute(df:Union[pl.DataFrame, pl.LazyFrame], cols: List[str], epsilon:float = 0.01) -> List[pl.Expr]:
    """
    Imputes columns by the min of the absolute values for c in columns, plus epsilon.
    """
    temp = df.lazy().select(pl.col(cols).abs().min() + epsilon).collect().row(0)
    return [pl.col(c).fill_null(m) for c, m in zip(cols, temp)]


In [ ]:
bp = (
    Blueprint(df, name = "example", target = "approved")
    .lowercase() # lowercase all columns
    .append_fit_func(smallest_abs_impute, ["var1", "existing_emi", "loan_amount"], epsilon = 0.5)
    # Use append_fit_func for custom transforms
)
# Notice that the value to impute is correct, it is 0.5, because the min abs of the columns are 0.
pipe:Pipeline = bp.materialize()
pipe

In [ ]:
pipe.transform(df).null_count().select(["var1", "existing_emi", "loan_amount"])